In [0]:
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

In [0]:
train_df = spark.read.csv('/FileStore/tables/train_features.csv', header='true', inferSchema=True)

In [0]:
train_df.printSchema()

root
-- sig_id: string (nullable = true)
-- cp_type: string (nullable = true)
-- cp_time: integer (nullable = true)
-- cp_dose: string (nullable = true)
-- g-0: double (nullable = true)
-- g-1: double (nullable = true)
-- g-2: double (nullable = true)
-- g-3: double (nullable = true)
-- g-4: double (nullable = true)
-- g-5: double (nullable = true)
-- g-6: double (nullable = true)
-- g-7: double (nullable = true)
-- g-8: double (nullable = true)
-- g-9: double (nullable = true)
-- g-10: double (nullable = true)
-- g-11: double (nullable = true)
-- g-12: double (nullable = true)
-- g-13: double (nullable = true)
-- g-14: double (nullable = true)
-- g-15: double (nullable = true)
-- g-16: double (nullable = true)
-- g-17: double (nullable = true)
-- g-18: double (nullable = true)
-- g-19: double (nullable = true)
-- g-20: double (nullable = true)
-- g-21: double (nullable = true)
-- g-22: double (nullable = true)
-- g-23: double (nullable = true)
-- g-24: double (nullable = true)
-- g-25: double (nullable = true)
-- g-26: double (nullable = true)
-- g-27: double (nullable = true)
-- g-28: double (nullable = true)
-- g-29: double (nullable = true)
-- g-30: double (nullable = true)
-- g-31: double (nullable = true)
-- g-32: double (nullable = true)
-- g-33: double (nullable = true)
-- g-34: double (nullable = true)
-- g-35: double (nullable = true)
-- g-36: double (nullable = true)
-- g-37: double (nullable = true)
-- g-38: double (nullable = true)
-- g-39: double (nullable = true)
-- g-40: double (nullable = true)
-- g-41: double (nullable = true)
-- g-42: double (nullable = true)
-- g-43: double (nullable = true)
-- g-44: double (nullable = true)
-- g-45: double (nullable = true)
-- g-46: double (nullable = true)
-- g-47: double (nullable = true)
-- g-48: double (nullable = true)
-- g-49: double (nullable = true)
-- g-50: double (nullable = true)
-- g-51: double (nullable = true)
-- g-52: double (nullable = true)
-- g-53: double (nullable = true)
-- g-54: double (nullable = true)
-- g-55: double (nullable = true)
-- g-56: double (nullable = true)
-- g-57: double (nullable = true)
-- g-58: double (nullable = true)
-- g-59: double (nullable = true)
-- g-60: double (nullable = true)
-- g-61: double (nullable = true)
-- g-62: double (nullable = true)
-- g-63: double (nullable = true)
-- g-64: double (nullable = true)
-- g-65: double (nullable = true)
-- g-66: double (nullable = true)
-- g-67: double (nullable = true)
-- g-68: double (nullable = true)
-- g-69: double (nullable = true)
-- g-70: double (nullable = true)
-- g-71: double (nullable = true)
-- g-72: double (nullable = true)
-- g-73: double (nullable = true)
-- g-74: double (nullable = true)
-- g-75: double (nullable = true)
-- g-76: double (nullable = true)
-- g-77: double (nullable = true)
-- g-78: double (nullable = true)
-- g-79: double (nullable = true)
-- g-80: double (nullable = true)
-- g-81: double (nullable = true)
-- g-82: double (nullable = true)
-- g-83: double (nullable = true)
-- g-84: double (nullable = true)
-- g-85: double (nullable = true)
-- g-86: double (nullable = true)
-- g-87: double (nullable = true)
-- g-88: double (nullable = true)
-- g-89: double (nullable = true)
-- g-90: double (nullable = true)
-- g-91: double (nullable = true)
-- g-92: double (nullable = true)
-- g-93: double (nullable = true)
-- g-94: double (nullable = true)
-- g-95: double (nullable = true)
-- g-96: double (nullable = true)
-- g-97: double (nullable = true)
-- g-98: double (nullable = true)
-- g-99: double (nullable = true)
-- g-100: double (nullable = true)
-- g-101: double (nullable = true)
-- g-102: double (nullable = true)
-- g-103: double (nullable = true)
-- g-104: double (nullable = true)
-- g-105: double (nullable = true)
-- g-106: double (nullable = true)
-- g-107: double (nullable = true)
-- g-108: double (nullable = true)
-- g-109: double (nullable = true)
-- g-110: double (nullable = true)
-- g-111: double (nullable = true)
-- g-112: double (nullable = true)
--

In [0]:
from pyspark.sql.functions import isnan, when, count, col
#check if there is missing value
train_df.select([count(when(isnan(c), c)).alias(c) for c in train_df.columns]).show()

+------+-------+-------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----

In [0]:
train_df.select("cp_type").distinct().show()

+-----------+
 cp_type|
+-----------+
 trt_cp|
ctl_vehicle|
+-----------+

In [0]:
train_df.select("cp_time").distinct().show()

+-------+
cp_time|
+-------+
 48|
 72|
 24|
+-------+

In [0]:
train_df.select("cp_dose").distinct().show()

+-------+
cp_dose|
+-------+
 D1|
 D2|
+-------+

In [0]:
enc_df = train_df.select("cp_type", "cp_time", "cp_dose")
enc_df.show(10)

+-------+-------+-------+
cp_type|cp_time|cp_dose|
+-------+-------+-------+
 trt_cp| 24| D1|
 trt_cp| 72| D1|
 trt_cp| 48| D1|
 trt_cp| 48| D1|
 trt_cp| 72| D2|
 trt_cp| 24| D1|
 trt_cp| 24| D2|
 trt_cp| 48| D1|
 trt_cp| 48| D1|
 trt_cp| 48| D2|
+-------+-------+-------+
only showing top 10 rows

In [0]:
from pyspark.ml.feature import StringIndexer

indexer_type = StringIndexer(inputCol="cp_type", outputCol="cp_type_ind")
indexer_time = StringIndexer(inputCol="cp_time", outputCol="cp_time_ind")
indexer_dose = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_ind")

indexed_type = indexer_type.fit(enc_df).transform(enc_df)
indexed_time = indexer_time.fit(enc_df).transform(enc_df)
indexed_dose = indexer_dose.fit(enc_df).transform(enc_df)

In [0]:
#join indexed columns
joined = indexed_type.join(indexed_time)
enc_df = joined.join(indexed_dose)
enc_df.select('cp_time_ind', 'cp_type_ind','cp_dose_ind')

Out[7]: DataFrame[cp_time_ind: double, cp_type_ind: double, cp_dose_ind: double]

In [0]:
enc_df.show(5)

In [0]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["cp_type_ind", "cp_time_ind", "cp_dose_ind"],
                        outputCols=["cp_type_vec", "cp_time_vec", "cp_dose_vec"])
model = encoder.fit(enc_df)
encoded = model.transform(enc_df)
encoded.show(5)